In [1]:
import pandas as pd
import os

In [2]:
# define path to read csvs
path_csv = "../data/"

# find all ".csv" files in the path
files = os.listdir(path_csv)
files = [f for f in files if f.endswith('.csv')]
print(files)

['watertemp_bop.csv', 'watertemp_gb.csv', 'watertemp_tasman.csv', 'waves_bop.csv', 'waves_gb.csv', 'waves_tb.csv', 'winds_bop.csv', 'winds_gb.csv', 'winds_tb.csv']


In [3]:
# read all csvs, saving each of them to a dataframe with the same name as the file
for f in files:
    name = f.split(".")[0]
    # replace winds with wind in the name if it exists

    name = name.replace("winds", "wind")
    # replace tasman with tb in the name
    name = name.replace("tasman", "tb")
    # replace watertemp with water in the name
    name = name.replace("watertemp", "water")

    # read csv
    globals()[name] = pd.read_csv(path_csv + f)
   
    # add two columns to the df: dataType and deploymentId. These two values are informed in the name of the file separated by "_"
    globals()[name]["dataType"] = name.split("_")[0] + "data"
    globals()[name]["region"] = name.split("_")[1].split(".")[0]

    
    # add the name of the dataframe to the list of dataframes
    if "dataframes" not in locals():
        dataframes = [name]
    else:
        dataframes.append(name)

In [4]:
import pytz

def _observationDate2Isoformat(dt):
    if dt.tzinfo is None:
        raise ValueError("Datetime object must be timezone-aware")

    # Convert to UTC timezone
    dt_utc = dt.astimezone(pytz.utc)

    # Format in ISO 8601 with milliseconds
    return dt_utc.isoformat()[:-6] + '.000Z'

In [8]:

# path where the processed dataframes will be saved
path_proc = "../warehouse/seeds/"

for df_name in dataframes:
    df = globals()[df_name]
    
    dataType = df["dataType"].unique()[0]
    df['time'] = pd.to_datetime(df['observationDate'], utc=True)
    
    # Extract fields based on field_mappings
    # times_water = [parse(item[df['time']]) for item in df]
    df['time'] = [_observationDate2Isoformat(i) for i in df['time']]

    # Define field mappings based on data type for the dataframes
    if dataType == 'waterdata':
        field_mappings = {
            'time': 'time',
            'temperature': 'value', # cannot be values
            'dataType': 'dataType',
            'region': 'region'
        }
    elif dataType == 'winddata':
        field_mappings = {
            'time': 'time',
            'windSpeed': 'speed',
            'windGust': 'gust',
            'windDirection': 'direction',
            'dataType': 'dataType',
            'region': 'region'
        }
    elif dataType == 'wavesdata':
        field_mappings = {
            'time': 'time',
            'significantWaveHeight': 'significantHeight',
            'maxWaveHeight': 'maxHeight',
            'meanWaveDirection': 'direction',
            'dataType': 'dataType',
            'region': 'region'
        }
    # Extract fields based on field_mappings and keep only the columns that are in the field_mappings
    df = df.rename(columns=field_mappings)
    df = df[list(field_mappings.values())]
    
    # save the processed dataframe to a csv file
    region = df["region"].unique()[0]
    dataType = df["dataType"].unique()[0]

    df.to_csv(path_proc + region + "_" + dataType + "_hist.csv", index=False)
